In [7]:
import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [3]:
nlp = spacy.load("fr_core_news_sm")
train_data = pd.read_csv("../archive/train.csv")
valid_data = pd.read_csv("../archive/valid.csv")
test_data  = pd.read_csv("../archive/test.csv")

train_reviews = train_data.review.values
test_reviews  = test_data.review.values


In [4]:
vectorizer = TfidfVectorizer(stop_words=list(fr_stop))
train_reviews_vectorised = vectorizer.fit_transform(train_reviews)
test_reviews_vectorised = vectorizer.transform(test_reviews)


/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


In [5]:
y_train = train_data['polarity']
y_test  = test_data['polarity']

## defining the model
model = LogisticRegression()

### MLFlow Tracking

In [ ]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    mlflow.sklearn.autolog(log_datasets=False)
    # Log the hyperparameters
    #mlflow.log_params(params)
    model.fit(train_reviews_vectorised,y_train)
    y_pred = model.predict(test_reviews_vectorised)
    # Log the loss metric
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall   = recall_score(y_test,y_pred)

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision",precision)


    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for review relevant")
    mlflow.set_tag("Preprocessing", "TF-IDF Vectorizer")
    #mlflow.source.name

    # Infer the model signature
    #signature = infer_signature(train_reviews_vectorised, model.predict(test_reviews_vectorised))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="Logistic_Regression v1",
        #signature=signature,
        registered_model_name="tracking-quickstart",
    )

2024/11/12 14:53:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/11/12 14:53:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.
2024/11/12 14:53:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run wise-swan-514 at: http://127.0.0.1:5000/#/experiments/714668274150591966/runs/ce5874e00b9e45ae900072ec8025d81f.
2024/11/12 14:53:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/714668274150591966.
